In [1]:
# magic command to display matplotlib plots inline within the ipython notebook webpage
%matplotlib inline

# import necessary modules
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import matplotlib.cm as cm, matplotlib.font_manager as fm
import datetime
import seaborn as sns
sns.set(color_codes=True)
import geopandas as gp
from scipy import stats, integrate

In [2]:
herepath = pd.read_csv('data/MOW_Path_Speed_5Min_2014_DOW_SpeedLimit.csv', encoding='utf-8')
herepath.head()

,LINK_DIR,DATE_TIME,EPOCH_5MIN,LENGTH,SPDLIMIT,MEAN,STDDEV,MIN,MAX,CONFIDENCE,PCT_5,PCT_50,PCT_85,DOW
0,998969850F,2014-01-01 13:55,167,186,72,61.0,0.0,61,61,10,61,61,61,Wednesday
1,998969850F,2014-01-02 11:05,133,186,72,60.0,0.0,60,60,10,60,60,60,Thursday
2,998969850F,2014-01-02 13:20,160,186,72,61.0,0.0,61,61,10,61,61,61,Thursday
3,998969850F,2014-01-02 14:30,174,186,72,52.0,0.0,52,52,10,52,52,52,Thursday
4,998969850F,2014-01-03 09:05,109,186,72,57.0,0.0,57,57,10,57,57,57,Friday


In [6]:
#For now, enter the weekdays desire here in full
Weekday = ['Monday','Tuesday','Wednesday','Thursday','Friday']
Weekday

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

In [7]:
Weekend = ['Saturday','Sunday']
Weekend

['Saturday', 'Sunday']

In [8]:
Both = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']
Both

['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [9]:
#w = Link, x = input dataframe, y = beginning timeframe (in epoch), z = ending timeframe (in epoch), 
#h = holiday (y or n), d = weekend or weekday
#Note the dataframe has to have an EPOCH_5MIN row to work.
#Also, this is ONLY for a single-link dataframe
def timeframe(w,x,y,z,h,d):
    us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']
    if h == True:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        r = x[x['DATE'].isin(us_holidays)]
        v = x[~x.isin(r)].dropna()
        link = v[v['LINK_DIR']== w]
        days = link[link['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe
    else:
        link = x[x['LINK_DIR']== w]
        days = link[link['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe

In [10]:
#w = Link, x = input dataframe, y = beginning timeframe (in epoch), z = ending timeframe (in epoch), 
#h = holiday (y or n), d = weekend or weekday
#Note the dataframe has to have an EPOCH_5MIN row to work.
#Also, this is ONLY for a single-link dataframe
def timeframe2(x,y,z,h,d):
    us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']
    if h == True:
        x['DATE'] = x['DATE_TIME'].str.split(' ').str[0]
        r = x[x['DATE'].isin(us_holidays)]
        v = x[~x.isin(r)].dropna()
        days = x[x['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe
    else:
        days = x[x['DOW'].isin(d)]
        Timeframe = days[(days['EPOCH_5MIN'] >= y) & (days['EPOCH_5MIN'] <= z)]
        return Timeframe

In [8]:
#Visualization on how the function works
us_holidays = ['2014-01-01', '2014-05-26', 
                   '2014-07-04', '2014-09-01', '2014-11-27', 
                   '2014-11-28', '2014-12-25']

herepath['DATE'] = herepath['DATE_TIME'].str.split(' ').str[0]
r = herepath[herepath['DATE'].isin(us_holidays)]
v = herepath[~herepath.isin(r)]
w = v.dropna()
r.head()

,Unnamed: 0,LINK_DIR,DATE_TIME,EPOCH_5MIN,LENGTH,MEAN,STDDEV,CONFIDENCE,DOW,DATE
0,0,998969850F,2014-01-01 13:55,167,186,61.0,0.0,10,Wednesday,2014-01-01
1099,1099,998969850F,2014-05-26 11:25,137,186,64.5,3.5,30,Monday,2014-05-26
1100,1100,998969850F,2014-05-26 14:15,171,186,76.0,0.0,10,Monday,2014-05-26
1101,1101,998969850F,2014-05-26 15:05,181,186,65.0,0.0,10,Monday,2014-05-26
1102,1102,998969850F,2014-05-26 17:25,209,186,31.0,0.0,10,Monday,2014-05-26


In [9]:
v.head()

,Unnamed: 0,LINK_DIR,DATE_TIME,EPOCH_5MIN,LENGTH,MEAN,STDDEV,CONFIDENCE,DOW,DATE
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,998969850F,2014-01-02 11:05,133.0,186.0,60.0,0.0,10.0,Thursday,2014-01-02
2,2.0,998969850F,2014-01-02 13:20,160.0,186.0,61.0,0.0,10.0,Thursday,2014-01-02
3,3.0,998969850F,2014-01-02 14:30,174.0,186.0,52.0,0.0,10.0,Thursday,2014-01-02
4,4.0,998969850F,2014-01-03 09:05,109.0,186.0,57.0,0.0,10.0,Friday,2014-01-03


In [10]:
#Proves the data doesn't have any NaN's other than what I specify to drop holidays
np.isnan(herepath.MEAN).unique()

array([False], dtype=object)

In [34]:
numberdays = data['DATE'].unique()
len(numberdays)

261

In [37]:
data = timeframe2(herepath,0,288,True,Weekday)
epoct = data['LINK_DIR'].value_counts()
epoct
epoct/(288*len(numberdays))

107030477T    0.221610
111990719T    0.213136
111990715T    0.212298
111999151F    0.208440
111990720F    0.200378
111990717F    0.197571
107030487T    0.195855
107034547T    0.193660
107030561T    0.192436
107373569T    0.192396
23320565T     0.188524
111996193T    0.182325
775719632T    0.182099
107030485T    0.181753
107373570F    0.181380
775719631T    0.180423
107030486T    0.179199
107030488F    0.178054
107030468T    0.177482
27593407F     0.177137
111986960F    0.173066
111999150F    0.170644
107030568T    0.170378
111989989T    0.168636
107030483F    0.168409
111990748T    0.167372
111990747T    0.165682
107030484F    0.165509
107030465T    0.161478
107030466T    0.161438
                ...   
111981912T    0.070855
997244941T    0.069444
23320410T     0.069325
107034534T    0.068952
997244940T    0.068034
27616976T     0.067688
27616977T     0.067436
107030432T    0.066824
111986235T    0.066544
111938004T    0.064083
23320387F     0.063471
111987140F    0.055901
107410437F 

In [ ]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsBase(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    z = pd.DataFrame(columns=[])
    
    z['PcntSpeed_85thAll'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    z['TotalIntervals_85thAll'] = epoct
    z['PcntInterval_85thAll'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    z['StdDev_85thAll'] = (np.sqrt(Var))*(0.6213)
    
   
    
    z['60spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.60,interpolation = 'lower'))*(0.6213)
    
    z['5spd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
      

    
    z = np.round(z,3)
    return z

In [15]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsALL(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    z = pd.DataFrame(columns=[])
    
    z['PcntSpeed_85thAll'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    z['TotalIntervals_85thAll'] = epoct
    z['PcntInterval_85thAll'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    z['StdDev_85thAll'] = (np.sqrt(Var))*(0.6213)
    
    z = np.round(z,3)
    return z

In [16]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWd(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    z = pd.DataFrame(columns=[])
    
    z['PcntSpeed_85thWd'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    z['TotalIntervals_85thWd'] = epoct
    z['PcntInterval_85thWd'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    z['StdDev_85thWd'] = (np.sqrt(Var))*(0.6213)
     
    z = np.round(z,3)
    return z

In [17]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWend(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    z = pd.DataFrame(columns=[])
    
    z['PcntSpeed_85thWend'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    z['TotalIntervals_85thWend'] = epoct
    z['PcntInterval_85thWend'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    z['StdDev_85thWend'] = (np.sqrt(Var))*(0.6213)
     
    z = np.round(z,3)
    return z

In [18]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdtime(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    z = pd.DataFrame(columns=[])
    
    z['PcntSpeed_60thWdtime'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.60,interpolation = 'lower'))*(0.6213)
    z['PcntSpeed_85thWdtime'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.85,interpolation = 'lower'))*(0.6213)
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    z['TotalIntervals_Wdtime'] = epoct
    z['PcntInterval_Wdtime'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    z['StdDev_Wdtime'] = (np.sqrt(Var))*(0.6213)
     
    z = np.round(z,3)
    return z

In [19]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayAM(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    z = pd.DataFrame(columns=[])
    
    z['PcntSpeed_5thWdayAM'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    z['AvgSpeed_WdayAM'] = y['MEAN']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    z['TotalIntervals_WdayAM'] = epoct
    z['PcntInterval_WdayAM'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    z['StdDev_WdayAM'] = (np.sqrt(Var))*(0.6213)
     
    z = np.round(z,3)
    return z

In [20]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayMD(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    z = pd.DataFrame(columns=[])
    
    z['PcntSpeed_5thWdayMD'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    z['AvgSpeed_WdayMD'] = y['MEAN']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    z['TotalIntervals_WdayMD'] = epoct
    z['PcntInterval_WdayMD'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    z['StdDev_WdayMD'] = (np.sqrt(Var))*(0.6213)
     
    z = np.round(z,3)
    return z

In [21]:
#ddof represents degrees of freedom which, according to the documentation for HERE, is 1
#85th percentile speed uses a linear interpolation.  The lowest, highest, or nearest can also be selected
def HEREstatsWdayPM(x):
    y = pd.DataFrame(columns=[])
    y['MEAN'] = ((x['MEAN']).groupby(x['LINK_DIR']).mean())*(0.6213)
    y['SPDLIMIT'] = (x['SPDLIMIT'].groupby(x['LINK_DIR']).mean())*(0.6213)
    
    z = pd.DataFrame(columns=[])
    
    z['PcntSpeed_5thWdayPM'] = (x['MEAN'].groupby(x['LINK_DIR']).quantile(0.05,interpolation = 'lower'))*(0.6213)
    z['AvgSpeed_WdayPM'] = y['MEAN']
    
    numberdays = x['DATE'].unique()
    epoct = x['LINK_DIR'].value_counts()
    z['TotalIntervals_WdayPM'] = epoct
    z['PcntInterval_WdayPM'] = ((epoct/(288*len(numberdays))*100))
    
    Var1 = (x['MEAN']).groupby(x['LINK_DIR'])
    Var2 = (x['STDDEV']).groupby(x['LINK_DIR'])
    Var = (Var1.var(ddof=1))+(Var2.sum()/len(x))
    z['StdDev_WdayPM'] = (np.sqrt(Var))*(0.6213)
     
    z = np.round(z,3)
    return z

In [10]:
def Indeces(y):
    y['TTI_SL'] = y['SPDLIMIT'].div(y['MEAN'])
    y['PTI_SL'] = y['SPDLIMIT'].div(y['5spd'])
    y['TTI_85th'] = y['85spd'].div(y['MEAN'])
    y['PTI_85th'] = y['85spd'].div(y['5spd'])
    y['TTI_60th'] = y['60spd'].div(y['MEAN'])
    y['PTI_60th'] = y['60spd'].div(y['5spd'])
    y['BTI'] = (y['MEAN'].div(y['5spd']))-1
    y = np.round(y,3)
    return y

In [98]:
#For all days and all time periods
bylink = HEREstatsALL(herepath)

bylink.head()

365


,MEAN,StDev,85spd,60spd,5spd,SPDLIMIT,TTI_SL,PTI_SL,TTI_85th,PTI_85th,TTI_60th,PTI_60th,BTI,TotalInterval,PcntInterval
LINK_DIR,,,,,,,,,,,,,,,
107030432T,38.553,10.444,48.461,42.248,19.260,49.704,1.289,2.581,1.257,2.516,1.096,2.194,1.002,5746,5.466
107030439T,35.148,9.929,44.734,38.521,17.396,44.734,1.273,2.571,1.273,2.571,1.096,2.214,1.020,7857,7.474
107030440T,34.863,10.303,45.355,38.521,17.086,44.734,1.283,2.618,1.301,2.655,1.105,2.255,1.040,7325,6.968
107030444F,36.180,11.209,47.219,40.695,16.154,44.734,1.236,2.769,1.305,2.923,1.125,2.519,1.240,9311,8.857
107030445F,36.602,11.250,47.840,41.627,16.775,44.734,1.222,2.667,1.307,2.852,1.137,2.481,1.182,9256,8.805


In [23]:
#for weekdays
Wkd = timeframe('998969850F',herepath,0,288,True,Weekday)
SpeedWd = HEREstatsWd(Wkd)
SpeedWd.head()

,PcntSpeed_85thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd
LINK_DIR,,,,
998969850F,42.87,2799,3.826,9.22


In [29]:
#for weekdays all links
Wkd = timeframe2(herepath,0,288,True,Weekday)
SpeedWd = HEREstatsWd(Wkd)
SpeedWd.head()

,PcntSpeed_85thWd,TotalIntervals_85thWd,PcntInterval_85thWd,StdDev_85thWd
LINK_DIR,,,,
107030432T,47.840,5023,6.682,10.344
107030439T,44.734,6775,9.013,9.937
107030440T,45.355,6339,8.433,10.262
107030444F,47.219,8045,10.703,11.301
107030445F,47.840,8008,10.653,11.282


In [24]:
#for weekends
Wend = timeframe('998969850F',herepath,0,288,True,Weekend)
SpeedWend = HEREstatsWend(Wend)
SpeedWend.head()

,PcntSpeed_85thAll,TotalIntervals_85thAll,PcntInterval_85thAll,StdDev_85thAll
LINK_DIR,,,,
998969850F,46.598,330,1.157,10.751


In [ ]:
#for weekends all links
Wend = timeframe2(herepath,0,288,True,Weekend)
SpeedWend = HEREstatsWend(Wend)
SpeedWend.head()

In [25]:
#Weekday Between 6AM and 8PM
Wdtime = timeframe('998969850F',herepath,72,240,True,Weekday)
SpeedWdtime = HEREstatsWdtime(Wdtime)
SpeedWdtime.head()

,PcntSpeed_60thWdtime,PcntSpeed_85thWdtime,TotalIntervals_Wdtime,PcntInterval_Wdtime,StdDev_Wdtime
LINK_DIR,,,,,
998969850F,36.035,41.627,2587,3.536,8.938


In [17]:
#Weekday Between 6AM and 8PM all links
Wdtime = timeframe2(herepath,72,240,True,Weekday)
SpeedWdtime = HEREstatsWdtime(Wdtime)
SpeedWdtime.head()

MemoryError: 

In [26]:
#Stats between 6AM and 9AM
WdtimeAM = timeframe('998969850F',herepath,72,108,True,Weekday)
SpeedWdayAM = HEREstatsWdayAM(WdtimeAM)
SpeedWdayAM.head()

,PcntSpeed_5thWdayAM,AvgSpeed_WdayAM,TotalIntervals_WdayAM,PcntInterval_WdayAM,StdDev_WdayAM
LINK_DIR,,,,,
998969850F,18.018,33.838,440,0.728,8.698


In [16]:
#Stats between 6AM and 9AM all links
WdtimeAM = timeframe2(herepath,72,108,True,Weekday)
SpeedWdayAM = HEREstatsWdayAM(WdtimeAM)
SpeedWdayAM.head()

261


134

In [27]:
#Stats between 9AM and 3PM
WdtimeMD = timeframe('998969850F',herepath,108,180,True,Weekday)
SpeedWdayMD = HEREstatsWdayMD(WdtimeMD)
SpeedWdayMD.head()

,PcntSpeed_5thWdayMD,AvgSpeed_WdayMD,TotalIntervals_WdayMD,PcntInterval_WdayMD,StdDev_WdayMD
LINK_DIR,,,,,
998969850F,18.639,34.014,1367,1.899,8.661


In [15]:
#Stats between 9AM and 3PM all links
WdtimeMD = timeframe2(herepath,108,180,True,Weekday)
SpeedWdayMD = HEREstatsWdayMD(WdtimeMD)
SpeedWdayMD.head()

261


134

In [28]:
#Stats between 3PM and 6PM
WdtimePM = timeframe('998969850F',herepath,180,216,True,Weekday)
SpeedWdayPM = HEREstatsWdayPM(WdtimeMD)
SpeedWdayPM.head()

,PcntSpeed_5thWdayPM,AvgSpeed_WdayPM,TotalIntervals_WdayPM,PcntInterval_WdayPM,StdDev_WdayPM
LINK_DIR,,,,,
998969850F,18.639,34.014,1367,1.899,8.661


In [14]:
#Stats between 3PM and 6PM all links
WdtimePM = timeframe2(herepath,180,216,True,Weekday)
SpeedWdayPM = HEREstatsWdayPM(WdtimeMD)
SpeedWdayPM.head()

261


134